Необхідні бібліотеки: urllib, pandas
(Також потрібно створити підкаталог "csv" для завантаження даних)

In [13]:
!pip install pandas
!pip install urllib 
!pip install IPython


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for urllib



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Завдання 1**

Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.
Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

Нижче написані дві процедури: 

Процедура обрахунку хеш-суми(для порівняння змісту файлів)

Процедура для завантаження файлу за індексом області, яка завантажує файли, вписує у назву дату та час завантаження. Завантаження файлу проходить в залежності того, чи немає вже цього файла у каталогу з даними.

In [14]:
import urllib.request
import hashlib
import os
from datetime import datetime

def calculate_file_hash(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

def download_data(index):
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=" + str(index) + "&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    now = datetime.now()
    date_and_time_time = now.strftime("%d%m%Y%H%M%S")
    new_file_path = f'./csv/noaa_id_{index}_{date_and_time_time}.csv'
    out = open(new_file_path,'wb')
    out.write(vhi_url.read())
    out.close()
    for filename in os.listdir('./csv/'):
        if filename.endswith(".csv") and f"id_{index}" in filename:
            existing_file_path = os.path.join('./csv/', filename)
            if existing_file_path != new_file_path:
                if calculate_file_hash(existing_file_path) == calculate_file_hash(new_file_path):
                    print(f"VHI{index} already exists")    
                    os.remove(new_file_path)
                    return
    print(f"VHI{index} is downloaded...")

for i in range(27):
    download_data(i+1)

VHI1 already exists
VHI2 already exists
VHI3 already exists
VHI4 already exists
VHI5 already exists
VHI6 already exists
VHI7 already exists
VHI8 already exists
VHI9 already exists
VHI10 already exists
VHI11 already exists
VHI12 already exists
VHI13 already exists
VHI14 already exists
VHI15 already exists
VHI16 already exists
VHI17 already exists
VHI18 already exists
VHI19 already exists
VHI20 already exists
VHI21 already exists
VHI22 already exists
VHI23 already exists
VHI24 already exists
VHI25 already exists
VHI26 already exists
VHI27 already exists


**Завдання 2**

Зчитати завантажені текстові файли у фрейм.
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття. Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

Нижче реалізована процедура, яка обробляє csv-файли: 

прибирає зайві символи

додає стовпець з індексом

об'єднує файли в один

прибирає дубльовані рядки

In [15]:
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

def concat_csv(directory):

    df_list = []
    words_to_remove = ["<br>", "<tt><pre>"]

    def remove_words(text):
        if isinstance(text, str):
            for word in words_to_remove:
                text = text.replace(word, "")
            return text.strip()
        return text

    sorted_filelist = sorted(os.listdir(directory), key=lambda x: int(x.split('_')[2]))

    for i in range(len(sorted_filelist)):
        filename = sorted_filelist[i]
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, index_col=False, header=1)
        headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
        df.columns = df.columns.str.replace("<br>", " ")
        df.columns = df.columns.str.strip()
        df['area'] = int(filename.split('_')[2])
        df = df.map(remove_words)
        df.drop(df[df["year"].str.contains("</pre></tt>", na=False)].index, inplace=True)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df_list.append(df)
        print(f"{filename} is added to main file")

    dfmain = pd.concat(df_list, ignore_index=True)
    dfmain = dfmain.drop_duplicates()
    dfmain.to_csv(f'./NOAA_Main.csv', index=False)
    
concat_csv('./csv/')

noaa_id_1_21022025232533.csv is added to main file
noaa_id_2_21022025232534.csv is added to main file
noaa_id_3_21022025232535.csv is added to main file
noaa_id_4_21022025232536.csv is added to main file
noaa_id_5_21022025232538.csv is added to main file
noaa_id_6_21022025232539.csv is added to main file
noaa_id_7_21022025232540.csv is added to main file
noaa_id_8_21022025232541.csv is added to main file
noaa_id_9_21022025232542.csv is added to main file
noaa_id_10_21022025232543.csv is added to main file
noaa_id_11_21022025232545.csv is added to main file
noaa_id_12_21022025232546.csv is added to main file
noaa_id_13_21022025232547.csv is added to main file
noaa_id_14_21022025232548.csv is added to main file
noaa_id_15_21022025232549.csv is added to main file
noaa_id_16_21022025232550.csv is added to main file
noaa_id_17_21022025232551.csv is added to main file
noaa_id_18_21022025232552.csv is added to main file
noaa_id_19_21022025232553.csv is added to main file
noaa_id_20_2102202523

**Завдання 3**

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) за українською:

Процедура змінює індекси за відповідними даними(відповідні індекси до областей та відповідності старих та нових назв)

In [16]:
ua_regions_eng = {
    1: "Cherkasy",
    2: "Chernihiv",
    3: "Chernivtsi",
    4: "Crimea",
    5: "Dnipropetrovs'k",
    6: "Donets'k",
    7: "Ivano-Frankivs'k",
    8: "Kharkiv",
    9: "Kherson",
    10: "Khmel'nyts'kyy",
    11: "Kiev",
    12: "Kiev City",
    13: "Kirovohrad",
    14: "Luhans'k",
    15: "L'viv",
    16: "Mykolayiv",
    17: "Odessa",
    18: "Poltava",
    19: "Rivne",
    20: "Sevastopol'",
    21: "Sumy",
    22: "Ternopil'",
    23: "Transcarpathia",
    24: "Vinnytsya",
    25: "Volyn",
    26: "Zaporizhzhya",
    27: "Zhytomyr"
}

ua_regions_ukr = {
    1: "Вінницька",
    2: "Волинська",
    3: "Дніпропетровська",
    4: "Донецька",
    5: "Житомирська",
    6: "Закарпатська",
    7: "Запорізька",
    8: "Івано-Франківська",
    9: "Київська",
    10: "Кіровоградська",
    11: "Луганська",
    12: "Львівська",
    13: "Миколаївська",
    14: "Одеська",
    15: "Полтавська",
    16: "Рівненська",
    17: "Сумська",
    18: "Тернопільська",
    19: "Харківська",
    20: "Херсонська",
    21: "Хмельницька",
    22: "Черкаська",
    23: "Чернівецька",
    24: "Чернігівська",
    25: "Республіка Крим",
    26: "Київ",
    27: "Севастополь"
}

ua_regions_compare = {
    "Cherkasy" : "Черкаська",
    "Chernihiv" : "Чернігівська",
    "Chernivtsi" : "Чернівецька",
    "Crimea" : "Республіка Крим",
    "Dnipropetrovs'k" : "Дніпропетровська",
    "Donets'k" : "Донецька",
    "Ivano-Frankivs'k" : "Івано-Франківська",
    "Kharkiv" : "Харківська",
    "Kherson" : "Херсонська",
    "Khmel'nyts'kyy" : "Хмельницька",
    "Kiev" : "Київська",
    "Kiev City" : "Київ",
    "Kirovohrad" : "Кіровоградська",
    "Luhans'k" : "Луганська",
    "L'viv" : "Львівська",
    "Mykolayiv" : "Миколаївська",
    "Odessa" : "Одеська",
    "Poltava" : "Полтавська",
    "Rivne" : "Рівненська",
    "Sevastopol'" : "Севастополь",
    "Sumy" : "Сумська",
    "Ternopil'" : "Тернопільська",
    "Transcarpathia" : "Закарпатська",
    "Vinnytsya" : "Вінницька",
    "Volyn" : "Волинська",
    "Zaporizhzhya" : "Запорізька",
    "Zhytomyr" : "Житомирська"
}

def change_regions_indexes(csvfilepath, csvnewfilepath, old, new, changetable):
    indexes_change = {}
    for old_index, old_name in old.items():
        new_name = changetable.get(old_name)
        if new_name:
            for new_index, new_region in new.items():
                if new_region == new_name:
                    indexes_change[old_index] = new_index
                    break
    csvfile = pd.read_csv(csvfilepath, index_col=False)
    csvfile["area"].replace(indexes_change, inplace = True)
    csvfile.to_csv(csvnewfilepath, index=False)
    print(f"{csvfilepath} has changed indexes in {csvnewfilepath}")

change_regions_indexes('./NOAA_Main.csv', './NOAA_Main_UAIndex.csv', ua_regions_eng, ua_regions_ukr, ua_regions_compare)

./NOAA_Main.csv has changed indexes in ./NOAA_Main_UAIndex.csv


**Завдання 4**

Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

o Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

o Ряд VHI за вказаний діапазон років для вказаних областей;

o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

Процедури для коректної роботи наступних процедур(також підключення бібліотеки для більш читабельного виводу даних)

In [17]:
from IPython.display import display
dfmain = pd.read_csv('./NOAA_Main_UAIndex.csv', index_col=False)

def area_to_index(area):
    if isinstance(area, str):
        index = [k for k, v in ua_regions_ukr.items() if v == area]
    elif isinstance(area, int):
        index = [area]
    return index[0]

Процедура для виводу ряду VHI(область, рік, тиждень та дані, що стосуються VHI) для вказаної області та року

Також процедура для перетворення вхідних областей на індекси

In [18]:
def vhi_for_area_year(df, area, year):
    current_df = df[(df["area"] == area_to_index(area)) & (df["year"] == year)]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"VHI data for {area} by {year}")
    print(f"---")
    display(current_df[['area', 'year', 'week', 'VCI', 'TCI', 'VHI']])
    

vhi_for_area_year(dfmain, "Донецька", 2006)

VHI data for Донецька by 2006
---


,area,year,week,VCI,TCI,VHI
12128,Донецька,2006,1.0,69.22,30.07,49.64
12129,Донецька,2006,2.0,71.43,36.44,53.94
12130,Донецька,2006,3.0,66.53,41.00,53.77
12131,Донецька,2006,4.0,57.27,51.22,54.25
12132,Донецька,2006,5.0,48.86,64.89,56.88
12133,Донецька,2006,6.0,38.28,76.76,57.52
12134,Донецька,2006,7.0,29.64,83.52,56.58
12135,Донецька,2006,8.0,24.31,88.88,56.60
12136,Донецька,2006,9.0,22.05,90.71,56.38
12137,Донецька,2006,10.0,23.13,90.02,56.58


Процедура для виводу ряду VHI з максимальним значенням VHI

In [19]:
def vhi_max_for_areas_years(df, areas, years):
    areaindexes = []
    for i in areas:
        areaindexes.append(area_to_index(i))
    current_df = df[(df["area"].isin(areaindexes)) & (df["year"].isin(years))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"Data with max VHI for {areas} by {years}")
    print(f"---")
    display(current_df[(current_df["VHI"] == current_df["VHI"].max())][['area', 'year', 'week', 'VCI', 'TCI', 'VHI']])

vhi_max_for_areas_years(dfmain, ["Донецька", "Київ"], [2004, 2006])

Data with max VHI for ['Донецька', 'Київ'] by [2004, 2006]
---


,area,year,week,VCI,TCI,VHI
12058,Донецька,2004,34.0,82.77,79.89,81.33


Процедура для виводу ряду VHI з мінімальним значенням VHI

In [20]:
def vhi_min_for_areas_years(df, areas, years):
    areaindexes = []
    for i in areas:
        areaindexes.append(area_to_index(i))
    current_df = df[(df["area"].isin(areaindexes)) & (df["year"].isin(years))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"Data with min VHI for {areas} by {years}")
    print(f"---")
    display(current_df[(current_df["VHI"] == current_df["VHI"].min())][['area', 'year', 'week', 'VCI', 'TCI', 'VHI']])

vhi_min_for_areas_years(dfmain, ["Донецька", "Київ"], [2004, 2006])

Data with min VHI for ['Донецька', 'Київ'] by [2004, 2006]
---


,area,year,week,VCI,TCI,VHI
12177,Донецька,2006,50.0,34.66,16.59,25.63


Процедура для виводу середнього значення VHI за обраними областями, роками

In [21]:
def vhi_average_for_areas_years(df, areas, years):
    areaindexes = []
    for i in areas:
        areaindexes.append(area_to_index(i))
    current_df = df[(df["area"].isin(areaindexes)) & (df["year"].isin(years))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"Average VHI for {areas} by {years}")
    print(f"---")
    print(current_df["VHI"].mean())

vhi_average_for_areas_years(dfmain, ["Донецька", "Київ"], [2004, 2006])

Average VHI for ['Донецька', 'Київ'] by [2004, 2006]
---
52.15720930232558


Процедура для виводу медіани VHI за обраними областями, роками

In [22]:
def vhi_median_for_areas_years(df, areas, years):
    areaindexes = []
    for i in areas:
        areaindexes.append(area_to_index(i))
    current_df = df[(df["area"].isin(areaindexes)) & (df["year"].isin(years))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"Median VHI for {areas} by {years}")
    print(f"---")
    print(current_df["VHI"].median())

vhi_median_for_areas_years(dfmain, ["Донецька", "Київ"], [2004, 2006])

Median VHI for ['Донецька', 'Київ'] by [2004, 2006]
---
52.845


Процедура для виводу ряду VHI за вказаний діапазон років для вказаних областей

In [23]:
def vhi_for_areas_between_years(df, areas, years):
    areaindexes = []
    for i in areas:
        areaindexes.append(area_to_index(i))
    current_df = df[(df["area"].isin(areaindexes)) & (df["year"].between(years[0], years[1]))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"VHI data for {areas} between {years[0]} and {years[1]}")
    print(f"---")
    display(current_df[['area', 'year', 'week', 'VCI', 'TCI', 'VHI']])

vhi_for_areas_between_years(dfmain, ["Донецька", "Київ"], [2004, 2006])

VHI data for ['Донецька', 'Київ'] between 2004 and 2006
---


,area,year,week,VCI,TCI,VHI
12043,Донецька,2004,1.0,45.00,76.10,60.55
12044,Донецька,2004,5.0,45.73,82.89,64.31
12045,Донецька,2004,6.0,42.72,84.22,63.47
12046,Донецька,2004,7.0,38.14,83.13,60.63
12047,Донецька,2004,8.0,36.50,80.41,58.46
...,...,...,...,...,...,...
25291,Київ,2006,48.0,43.89,15.38,29.64
25292,Київ,2006,49.0,43.10,13.43,28.26
25293,Київ,2006,50.0,43.50,13.94,28.72
25294,Київ,2006,51.0,45.60,16.01,30.81


Процедура для виводу років, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні(20% за замовчуванням), а також виводу даних за цими роками(роки, області, VHI)

In [24]:
def extreme_drought_years(df, percent=0.2):
    total_areas = df["area"].unique()
    total_years = df["year"].unique()
    critical_vhi = 15  
    affected_limit = int(len(total_areas) * percent)
    extreme_years = {}
    totaldroughtareas = []
    for i in total_years:
        droughts = df[(df['year'] == i) & (df['VHI'] <= critical_vhi)]
        droughtareas = droughts["area"].unique()
        if len(droughtareas) >= affected_limit:
            extreme_years[int(i)] = droughtareas
            for i in droughtareas:
                totaldroughtareas.append(i)
    print(f"Extreme years(where VHI <= 15): {list(extreme_years.keys())}")
    print(f"---")
    current_df = df[(df["year"].isin(list(extreme_years.keys()))) & (df['area'].isin(totaldroughtareas))]
    current_df["area"].replace(ua_regions_ukr, inplace = True)
    print(f"VHI data by {list(extreme_years.keys())}")
    print(f"---")
    display(current_df[["year", "area", "VHI"]])

extreme_drought_years(dfmain, percent=0.1)

Extreme years(where VHI <= 15): [1986, 1993, 2000, 2007]
---
VHI data by [1986, 1993, 2000, 2007]
---


,year,area,VHI
197,1986,Черкаська,53.55
198,1986,Черкаська,53.61
199,1986,Черкаська,50.22
200,1986,Черкаська,46.11
201,1986,Черкаська,44.93
...,...,...,...
55947,2007,Запорізька,43.54
55948,2007,Запорізька,41.30
55949,2007,Запорізька,40.07
55950,2007,Запорізька,39.34
